## dynamo: RNA velocity, vector field reconstruction and dynamics, and more
This script notebook tests a few tools of the dynamo framework on an scRNASeq dataset.  
dynamo documentation: https://dynamo-release.readthedocs.io/en/latest/  
dynamo paper: https://www.sciencedirect.com/science/article/pii/S0092867421015774  
dynamo tutorial (vector field reconstruction): https://dynamo-release.readthedocs.io/en/latest/notebooks/zebrafish.html  
dynamo tutorial (differential geometry tools): https://dynamo-release.readthedocs.io/en/latest/notebooks/Differential_geometry.html  
The framework is still in development, so errors and/or dependency conflicts could appear.  

The working directory is Ahmad_workdir

In [ ]:
!pip install KDEpy
!git clone https://github.com/aristoteleo/dynamo-release.git
!pip install dynamo-release/ --user
!pip install nxviz==0.7.3
!pip install yt

In [ ]:
import dynamo as dyn
dyn.configuration.set_figure_params('dynamo', background='white') # jupter notebooks

In [ ]:
# Pipeline using scvelo loom object

# import precomputed vaevictis embeddings (see also rna_velocity_final.ipynb)
import pandas as pd

full_vae = pd.read_csv('seurat_mice_out/mice_all_red2.csv', header=None)
full_vae = full_vae.set_index(keys=[0])
names = full_vae.index.values.tolist()
names = [c.split('-')[0] for c in names]
full_vae.index = names

# read loom file output of velocyto
import scvelo as scv
adataT = scv.read('scvelo_out/ALL_T/sample_alignments_merged_5KJ8Z.loom', cache=True)
adataT.var_names_make_unique()

# fix name formatting and add vaevictis embeddings
all_brcd = adataT.obs_names.values.tolist()
all_brcd = [c.split(':')[-1][:-1] for c in all_brcd]
adataT.uns['vae'] = {}
full_vae_u = full_vae[~full_vae.index.duplicated(keep='first')]
adataT.obsm['X_vae'] = full_vae_u.loc[all_brcd].values

In [ ]:
dyn.pp.recipe_monocle(adataT) # Preprocess
dyn.tl.dynamics(adataT) # Learn dynamics
dyn.tl.reduceDimension(adataT) # Reduce dim
dyn.tl.cell_velocities(adataT, basis='vae') # Evaluate velocity vectors
dyn.tl.cell_wise_confidence(adataT) # Evaluate cell-wise velocity confidence
dyn.vf.VectorField(adataT, basis='vae') # Reconstruct vector field from learned velocity vectors
dyn.vf.topography(adataT, basis='vae') # Characterize vector field topology
dyn.ext.ddhodge(adataT, basis='vae') # Decompose vector field
# dyn.vf.Potential(adataT) # Map potential landscape (function does not work as of September)

In [ ]:
# Visualization
dyn.pl.cell_wise_vectors(adataT, basis='vae', ncols=3)
# dyn.pl.grid_vectors(adataT, basis='vae', ncols=3)
dyn.pl.streamline_plot(adataT, basis='vae', ncols=3, density=2)
dyn.pl.line_integral_conv(adataT, basis='vae')
dyn.pl.topography(adataT, basis='vae', fps_basis='vae', init_cells=[675, 1788])

In [ ]:
dyn.pl.topography(adataT, basis='vae', fps_basis='vae')
# no difference was observed if the root cells are specified vs if they are not

In [ ]:
transition_genes = adataT.var_names[adataT.var.use_for_transition]
dyn.pl.streamline_plot(adataT, color='vae_ddhodge_potential', basis='vae', density=2.5)

In [ ]:
# Vector field analysis for diffgeo tools
dyn.vf.speed(adataT, basis='vae')
dyn.vf.divergence(adataT, basis='vae')
dyn.vf.acceleration(adataT, basis='vae')
dyn.vf.curvature(adataT, basis='vae')
dyn.vf.curl(adataT, basis='vae')

In [ ]:
dyn.pl.streamline_plot(adataT, color='curvature_vae', basis='vae')

In [ ]:
dyn.pl.streamline_plot(adataT, color='curl_vae', basis='vae')

In [ ]:
# Compute Jacobian
# Alleviate computation by taking top genes
dyn.pp.top_pca_genes(adataT, n_top_genes=100)
top_pca_genes = adataT.var.index[adataT.var.top_pca_genes]
dyn.vf.jacobian(adataT, regulators=top_pca_genes, effectors=top_pca_genes)